# Map-2-JSON

This is the translation of an older [Gist](https://gist.github.com/rdmueller/793d262862270eda8d3a0b5c2e9e46da) (which was inspired by jupyter) to a jupyter notebook with [beakerx](https://github.com/twosigma/beakerx) and [binder](https://mybinder.org/).



## Problem: 

Languages like tcl can simply switch between the String representation of a map or list and the object itself. In Groovy, this could theoretically work, but it doesn't:

In [1]:
def list1 = [1,2,3]
def list2 = ["1, 2, 3"]
assert         list1             !=         list2
assert         list1.toString()  ==         list2.toString()
assert Eval.me(list1.toString()) instanceof List
assert Eval.me(list2.toString()) instanceof List
assert Eval.me(list1.toString()) == Eval.me(list2.toString())

null

## Solution:
Groovy has several ways to serialize a map or list. One popular way it to convert it to JSON
http://groovy-lang.org/json.html

In [1]:
import groovy.json.*
map = [
            list:[1,2,3],
            integer: 5,
            bigDecimal: 3.1415,
            string: "yes",
            boolean: true,
            //date: new Date(0)
      ]
mapAsJson = JsonOutput.toJson(map)
assert mapAsJson == '{"list":[1,2,3],"integer":5,"bigDecimal":3.1415,"string":"yes","boolean":true}'
assert new JsonSlurper().parseText(mapAsJson) instanceof Map
assert new JsonSlurper().parseText(mapAsJson) == map

null

as you can see, the entry with type "Date" is commented out. Unfortunately, the conversion isn't bi-directional for all data types

another popular way to serialze a map is YAML. YAML has the slight drawback that it needs an external library:

In [2]:
@Grab("org.yaml:snakeyaml:1.16")
import org.yaml.snakeyaml.Yaml

def mapAsYaml = new Yaml().dump(map)
assert mapAsYaml == """list: [1, 2, 3]
integer: 5
bigDecimal: 3.1415
string: 'yes'
boolean: true
"""
assert new Yaml().load(mapAsYaml) instanceof Map
assert new Yaml().load(mapAsYaml) == map

null

Both ways work fine. The main difference is that the String representation of JSON is one line by default where YAML is distributed over several lines.

In order to make these serializations easier to use, we can make use of metaprogramming:

In [3]:
import groovy.json.*
    
String.metaClass.toMap {->
    new JsonSlurper().parseText(delegate)
}
assert '{"list":[1,2,3],"integer":5,"bigDecimal":3.1415,"string":"yes","boolean":true}'.toMap() == map


null

Unfortunately, the toString() can't be overwritten:

In [4]:
Map.metaClass.toString {->
    JsonOutput.toJson(delegate)
}
assert '{"list":[1,2,3],"integer":5,"bigDecimal":3.1415,"string":"yes","boolean":true}'         != map.toString()


null

...but a toJson() method can be created:

In [5]:
import groovy.json.*
    
Map.metaClass.toJson {->
    JsonOutput.toJson(delegate)
}
assert '{"list":[1,2,3],"integer":5,"bigDecimal":3.1415,"string":"yes","boolean":true}'         == map.toJson()


null

//btw: since YAML is a superset of JSON, you can deserialize JSON with YAML:


In [8]:
import org.yaml.snakeyaml.Yaml

assert new Yaml().load(mapAsJson) == map


null

//but not the other way around.

//Groovy version used: 2.4.5